# Run jobs in a batch

Use batch mode to submit multiple primitive jobs simultaneously. Following are examples of working with batches.

## Set up to use batches

Before starting a batch, you must [set up Qiskit Runtime](/docs/guides/install-qiskit) and initialize it as a service:

In [1]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)


service = QiskitRuntimeService()

## Open a batch

You can open a runtime batch by using the context manager `with Batch(...)` or by initializing the `Batch`
class. When you start a batch,  you must specify a QPU by passing a `backend` object.  The batch starts when its first job begins execution.


**Batch class**

In [2]:
backend = service.least_busy(operational=True, simulator=False)
batch = Batch(backend=backend)
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
# Close the batch because no context manager was used.
batch.close()

**Context manager**

The context manager automatically opens and closes the batch.

In [3]:
from qiskit_ibm_runtime import (
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)

backend = service.least_busy(operational=True, simulator=False)
with Batch(backend=backend):
    estimator = Estimator()
    sampler = Sampler()

<span id="specify-batch-length"></span>
## Batch length

You can define the batch's maximum time to live (TTL) with the `max_time` parameter. This should exceed the longest job's execution time. This timer starts when the batch starts.  When the value is reached, the batch is closed.  Any jobs that are running will finish, but jobs still queued are failed.

```python
with Batch(backend=backend, max_time="25m"):
  ...
```

There is also an interactive time to live (interactive TTL) value that cannot be configured (1 minute for all plans).  If no batch jobs are queued within that window, the batch is temporarily deactivated.

Default maximum TTL values:

| Instance type  | Default Maximum TTL |
|----------------|---------------------|
| All paid plans | 8 hours             |
| Open           | 10 minutes          |

To determine a batch's maximum TTL or interactive TTL, follow the instructions in [Determine batch details](#batch-details) and look for the `max_time` or `interactive_timeout` value, respectively.

<span id="close"></span>
## Close a batch

A batch automatically closes when it exits the context manager. When the batch context manager is exited, the batch is put into "In progress, not accepting new jobs" status. This means that the batch finishes processing all running or queued jobs until the maximum TTL value is reached. After all jobs are completed, the batch is immediately closed. You cannot submit jobs to a closed batch.

In [4]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.transpiler import generate_preset_pass_manager
import numpy as np

# This cell is hidden from users
service = QiskitRuntimeService()
backend = service.least_busy()

# Define two circuits, each with one parameter with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.cx(0, 1)
circuit.h(0)
circuit.measure_all()


pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
transpiled_circuit_sampler = transpiled_circuit
transpiled_circuit_sampler.measure_all()

params = np.random.uniform(size=(2, 3)).T
observables = [
    [
        SparsePauliOp(["XX", "IY"], [0.5, 0.5]).apply_layout(
            transpiled_circuit.layout
        )
    ],
    [SparsePauliOp("XX").apply_layout(transpiled_circuit.layout)],
    [SparsePauliOp("IY").apply_layout(transpiled_circuit.layout)],
]

sampler_pub = (transpiled_circuit_sampler, params)
estimator_pub = (transpiled_circuit_sampler, observables, params)

In [5]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])

# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

<Admonition type="tip">
If you are not using a context manager, manually close the batch.  If you leave the batch open and submit more jobs to it later, it is possible that the maximum TTL will be reached before the subsequent jobs start running; causing them to be canceled. You can close a batch as soon as you are done submitting jobs to it. When a batch is closed with `batch.close()`, it no longer accepts new jobs, but the already submitted jobs will still run until completion and their results can be retrieved.
</Admonition>

In [6]:
batch = Batch(backend=backend)

# If using qiskit-ibm-runtime earlier than 0.24.0, change `mode=` to `batch=`
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")

# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

Result1: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(3, 2), dtype=float64>), stds=np.ndarray(<shape=(3, 2), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(3, 2), dtype=float64>), shape=(3, 2)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
Result2: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(3, 2), num_shots=4096, num_bits=2>), meas0=BitArray(<shape=(3, 2), num_shots=4096, num_bits=127>), shape=(3, 2)), metadata={'circ

<span id="batch-details"></span>
## Determine batch details

For a comprehensive overview of a batch's configuration and status, including its interactive and max TTL, use the  `batch.details() method`.

In [7]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    batch,
    SamplerV2 as Sampler,
)

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

with Batch(backend=backend) as batch:
    print(batch.details())

{'id': '1c7e6c9a-9ab3-4eb9-934e-f96f568d92e6', 'backend_name': 'ibm_pinguino2', 'interactive_timeout': 1, 'max_time': 86400, 'active_timeout': 14400, 'state': 'open', 'accepting_jobs': True, 'last_job_started': None, 'last_job_completed': None, 'started_at': None, 'closed_at': None, 'activated_at': None, 'mode': 'batch', 'usage_time': None}


<span id="partition"></span>
## Reconfigure jobs for parallel processing

There are multiple ways you can reconfigure your jobs to take advantage of the parallel processing provided by batching. The following example shows how you can partition a long list of circuits into multiple jobs and run them as a batch to take advantage of the parallel processing.

In [31]:
from qiskit_ibm_runtime import SamplerV2 as Sampler, Batch
from qiskit.circuit.random import random_circuit

max_circuits = 100
circuits = [pm.run(random_circuit(5, 5)) for _ in range(5 * max_circuits)]
for circuit in circuits:
    circuit.measure_active()
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits):
    all_partitioned_circuits.append(circuits[i : i + max_circuits])
jobs = []
start_idx = 0

with Batch(backend=backend):
    sampler = Sampler()
    for partitioned_circuits in all_partitioned_circuits:
        job = sampler.run(partitioned_circuits)
        jobs.append(job)

<Admonition type="caution">
    If you set `backend=backend` in a primitive, the program is run in job mode, even if it's inside a batch or session context. Setting `backend=backend` is deprecated as of Qiskit Runtime 0.24.0.  Instead, use the `mode` parameter.
</Admonition>

## Next steps

<Admonition type="tip" title="Recommendations">
  - Try an example in the [Combine error mitigation options with the estimator primitive](/docs/tutorials/combine-error-mitigation-techniques) tutorial.
  - Review the [Batch API](/docs/api/qiskit-ibm-runtime/batch#batch) reference.
  - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM QPU.
</Admonition>